# COMP5329 ASSIGNMENT - EXPERIMENTS


## INSTRUCTIONS:
* Run cells in order to untile section 'Models'
* Choose Desired Model or choose 'Load Model and Weights' to use final model and weights
* Run 'Compile Model' cell and Training cells
* Run all cells below that point to generate classification report and confusion matrix

### Import Relevant Packages

In [2]:
# --------------
# BASIC PACKAGES
# --------------
%matplotlib inline
import numpy as np
seed = 99
np.random.seed(99)
import matplotlib.pyplot as plt
import scipy as sp
import time
from datetime import timedelta
import math
import imageio
import os
import itertools

# ---------------------------
# TENSORFLOW & KERAS PACKAGES
# ---------------------------
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import model_from_json
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from sklearn.metrics import classification_report

# ------------------------------------
# SCIKIT-LEARN PACKAGES FOR EVALUATION
# ------------------------------------
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Import Labels and Shuffle

In [4]:
# (37882, 2)
train_lbls = np.loadtxt('train.txt', dtype=str, delimiter=' ')
# (6262, 2)
vali_lbls = np.loadtxt('vali.txt', dtype=str, delimiter=' ')

In [5]:
np.random.seed(seed)
np.random.shuffle(train_lbls)
np.random.seed(seed)
np.random.shuffle(vali_lbls)

### Import Training and Validation Sets (Rescale data to between 0 and 1)

In [6]:
def READ_IN_IMGS(X_train_folder, X_labels, img_height, img_width):
    
    start = time.time()
    no_of_samples = X_labels.shape[0]
    X_train_array = np.zeros((no_of_samples,img_height,img_width))
    j=0
    for i in X_labels:
        img_ary = imageio.imread(X_train_folder+i)
        img_ary = img_ary.reshape((1,) + img_ary.shape) 
        X_train_array[j] = img_ary
        if j < X_train_array.shape[0]:
            j+=1
    X_train_array = X_train_array.astype('float32')
    X_train_array = X_train_array / 255
    print(time.time() - start)
    
    return(X_train_array)

In [7]:
X_train = READ_IN_IMGS('train-set/',train_lbls[:,0],128,128)
X_vali = READ_IN_IMGS('vali-set/',vali_lbls[:,0],128,128)

19.43223476409912
3.180396318435669


In [8]:
# RAW IMAGES
X_train[3,45:,45:]

array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

### Normalize the mean around zero for range -1 to 1

In [9]:
# X_train -= 0.5
# X_vali -= 0.5
# X_train *= 2.
# X_vali *= 2.

In [10]:
# X_train[3,45:,45:]

In [11]:
X_train = X_train.reshape(X_train.shape[0], 128, 128, 1)
X_vali = X_vali.reshape(X_vali.shape[0], 128, 128, 1)

In [12]:
X_train.shape

(37882, 128, 128, 1)

### Convert Labels to One Hot Encoding

In [13]:
y_vali = np_utils.to_categorical(vali_lbls[:,1], 62)
y_train = np_utils.to_categorical(train_lbls[:,1], 62)

In [14]:
class_num = y_vali.shape[1]

# Experimental Models

### (0) Simple Neural Network

In [15]:
model = Sequential()

model.add(Flatten(input_shape=X_train.shape[1:]))
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024)) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256)) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(62))
model.add(BatchNormalization())
model.add(Activation('softmax'))

### (A) CNN3232_6464_128128_FC4096F4096_drpt_BN

In [21]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))
print(model.output_shape)
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
print(model.output_shape)

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# model.add(Conv2D(512, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(512, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

print(model.output_shape)
model.add(Flatten())
print(model.output_shape)

model.add(Dense(4096)) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4096)) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(class_num))
model.add(BatchNormalization())
model.add(Activation('softmax'))

(None, 128, 128, 32)
(None, 63, 63, 32)
(None, 14, 14, 128)
(None, 25088)


### (B/C) CNN3232_3232_3232_FC1024F1024_drpt_BN

In [19]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))
print(model.output_shape)
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
print(model.output_shape)

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# model.add(Conv2D(32, (3, 3), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Conv2D(32, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(32, (3, 3), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Conv2D(32, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

print(model.output_shape)
model.add(Flatten())
print(model.output_shape)

model.add(Dense(1024)) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024)) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(class_num))
model.add(BatchNormalization())
model.add(Activation('softmax'))

(None, 128, 128, 32)
(None, 63, 63, 32)
(None, 14, 14, 32)
(None, 6272)


### (D) CNN3232_3232_3232_FC512F512_drpt_BN(momentum=0.6)

In [23]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
print(model.output_shape)
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
print(model.output_shape)

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

print(model.output_shape)
model.add(Flatten())
print(model.output_shape)

model.add(Dense(512)) 
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512)) 
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(class_num))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('softmax'))

(None, 128, 128, 32)
(None, 63, 63, 32)
(None, 14, 14, 32)
(None, 6272)


### (E) CNN3232_3232_3232_3232_FC512F512_drt_BN(momentum=0.6)

In [27]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
print(model.output_shape)
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
print(model.output_shape)

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

print(model.output_shape)
model.add(Flatten())
print(model.output_shape)

model.add(Dense(512)) 
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512)) 
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(class_num))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('softmax'))

(None, 128, 128, 32)
(None, 63, 63, 32)
(None, 6, 6, 32)
(None, 1152)


### (F) CNN3232_3232_3232_FC512F512FC512FC62_dt_BN(mtm=0.6)

In [32]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
print(model.output_shape)
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
print(model.output_shape)

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

print(model.output_shape)
model.add(Flatten())
print(model.output_shape)

model.add(Dense(512)) 
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512)) 
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512)) 
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(62)) 
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(class_num))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('softmax'))

(None, 128, 128, 32)
(None, 63, 63, 32)
(None, 14, 14, 32)
(None, 6272)


### (G) CNN32_32_32_32_32_FC512F512_drpt_BN(momentum=0.6)

In [29]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
print(model.output_shape)
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
print(model.output_shape)

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# model.add(Conv2D(32, (3, 3), padding='same'))
# model.add(BatchNormalization(momentum=0.6))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

print(model.output_shape)
model.add(Flatten())
print(model.output_shape)

model.add(Dense(512)) 
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512)) 
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(class_num))
model.add(BatchNormalization(momentum=0.6))
model.add(Activation('softmax'))

(None, 128, 128, 32)
(None, 64, 64, 32)
(None, 4, 4, 32)
(None, 512)


### Load Model and Weights

In [3]:
json_file = open("Model_D_model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("ModelD_weights.hdf5")

### Data Augmentation Train/Test Split

In [ ]:
# X_train_dataug, X_test_dataug, y_train_dataug, y_test_dataug = train_test_split(X_train, y_train, test_size=0.1, random_state=99)

### Define Data Augmentation Parameters

In [ ]:
# datagen = ImageDataGenerator(width_shift_range=0.2,
#                             height_shift_range=0.2)

In [ ]:
# datagen = ImageDataGenerator(zoom_range=0.25)

In [ ]:
# Halve the size of input image to speed up training?
# datagen = ImageDataGenerator(rescale=0.5)

### Preprocessing: ZCA Whitening 

In [ ]:
# datagen = ImageDataGenerator(zca_whitening=True)
# Defualt: zca_epsilon=1e-06,

### Fit Data Augmentation to X_train Split

In [ ]:
# ZCA Whitening - Time to Execute: 19 minutes on AWS
# datagen.fit(X_train_dataug)

In [ ]:
# os.makedirs('zca_1e06')

### Compile Model

In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
# checkpoint = ModelCheckpoint("model_weights_BEST.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
checkpoint = ModelCheckpoint("weights.{epoch:02d}-{val_acc:.2f}.hdf5", monitor='val_acc', verbose=1)

In [23]:
csv_logger = CSVLogger('5329training.log')

### Fit Model to Augmented Dataset

In [ ]:
# CNN_model = model.fit_generator(datagen.flow(X_train_dataug, y_train_dataug, batch_size=32),
#                     steps_per_epoch=len(X_train) / 32, epochs=30,
#                     callbacks=[csv_logger, checkpoint], 
#                     validation_data = (X_test_dataug, y_test_dataug))

In [ ]:
# ZCA Whitening
# Epoch 1/20
# 1184/1183 [==============================] - 2065s 2s/step - loss: 1.7467 - acc: 0.6089 - val_loss: 4.6675 - val_acc: 0.0264

### Fit Model to Original Dataset

In [24]:
test_model = model.fit(X_train, y_train,
                  batch_size=32,
                  epochs=30, 
                    callbacks=[csv_logger, checkpoint], 
                    validation_data = (X_vali,y_vali))

Train on 37882 samples, validate on 6262 samples
Epoch 1/30
37882/37882 [==============================] - 126s 3ms/step - loss: 1.7376 - acc: 0.6146 - val_loss: 0.6509 - val_acc: 0.8197

Epoch 00001: saving model to weights.01-0.82.hdf5
Epoch 2/30
37882/37882 [==============================] - 123s 3ms/step - loss: 1.0063 - acc: 0.7465 - val_loss: 0.5007 - val_acc: 0.8499

Epoch 00002: saving model to weights.02-0.85.hdf5
Epoch 3/30
37882/37882 [==============================] - 123s 3ms/step - loss: 0.8074 - acc: 0.7785 - val_loss: 0.4522 - val_acc: 0.8638

Epoch 00003: saving model to weights.03-0.86.hdf5
Epoch 4/30
37882/37882 [==============================] - 123s 3ms/step - loss: 0.6983 - acc: 0.7998 - val_loss: 0.4032 - val_acc: 0.8711

Epoch 00004: saving model to weights.04-0.87.hdf5
Epoch 5/30
37882/37882 [==============================] - 123s 3ms/step - loss: 0.6171 - acc: 0.8168 - val_loss: 0.3990 - val_acc: 0.8719

Epoch 00005: saving model to weights.05-0.87.hdf5
Epoch 

In [33]:
# model.load_weights("weights.22-0.91.hdf5")

### Evaluate Model on Holdout Testset

In [34]:
scores = model.evaluate(X_vali, y_vali, verbose=1)
scores

6262/6262 [==============================] - 7s 1ms/step


[0.2794214840544104, 0.9078569146666197]

### Model Experiments Results:

In [ ]:
#Baseline Preprocessing: value / 255
# Adam optimizer

# MODEL ZERO - NN
# NN_4096_1024_256_62_dropt_BN0.99 97s e, 86% @ 20 epoch converged

# MODEL A - VGGNet-Like
# CNN3232_6464_128128_FC4096F4096_drpt_BN0.99 297s e, 88 valacc @ 5e (not o/fit yet) - too complex? run 25e

# MODEL B - Less Layer Width (simpler and faster)
# CNN3232_3232_3232_FC1024F1024_drpt_BN0.99 115s e, 90valcc @ 15 epochs then o/fit (ran 20e) 
# [0.36212299809369064, 0.8829447460304012]

# MODEL C - Less Layer Width with BN momentum = 0.6
# CNN3232_3232_3232_FC1024F1024_drpt_BN0.6 115s e, 89.9 @ 12 epochs (stopped @ 20 epoch)
# [loss:0.3085, acc:0.9071] test acc after 20 epochs
# Epoch 20/20
# 34093/34093 [==============================] - 115s 3ms/step - loss: 0.2349 - acc: 0.9197 - val_loss: 0.3097 - val_acc: 0.9111

# MODEL D - faster - same result (not overfit yet) but loss converged @ 12e
# CNN3232_3232_3232_FC512F512_drpt_BN(momentum=0.6) 109s e, 90.76 @ 20 epochs (not overfit yet)
# holdout test result: [loss:0.3006, acc:0.9064]
# MODEL D(a) - Mean centred at 0, rang -1 to 1 40 epochs
# CNN3232_3232_3232_FC512F512_drpt_BN0.6 110s e, 90.92 @ 25 epochs then overfit
# @39e acc: 0.9317, val_acc: 0.9137, hlodout test: [0.296, 0.9115]

# MODEL D(b) - (Model D with width_shift_range=0.2,height_shift_range=0.2 - lower - not converged but plateaued - potentially warping images horiz/vertically dispropotionatily?
# CNN3232_3232_3232_FC512F512_drpt_BN0.6 122s e, 87 @ 30 epochs 
# holdout test result: [0.3712, 0.8708]

# MODEL D(c) - (Model D with zoom_range=0.25 0-mean, (-1,1)- Worse h/out result than D- not converged yet but flat @ 30e
# CNN3232_3232_3232_FC512F512_drpt_BN0.6 122s e, 91 @ 30 epochs 
# holdout test result: [0.2877, 0.8876]

# MODEL E - not converged yet - run more epochs
# CNN_3232_3232_3232_3232_FC512F512_adam_drop_BN0.6 110s e, acc 80.5, valacc89 @ 20 epochs 
# holdout test result: [0.2960, 0.8978]

# MODEL F - (Model D with ZCA Whitening)
# Epoch 1/20
# 1184/1183 [==============================] - 2065s 2s/step - loss: 1.7467 - acc: 0.6089 - val_loss: 4.6675 - val_acc: 0.0264

# MODEL F - More FC layers 
# CNN3232_3232_3232_FC512F512FC512FC62_drpt_BN(momentum=0.6) 114s e, 85 @ 30 epochs
# [0.402, 0.8464]

# MODEL G - 5 x small conv2d then maxpoool - simpler but deep - not converged @ 100e SAVED WEIGHTS
# CNN32_32_32_32_32_FC512F512_drpt_BN0.6 53s e, 0.89997 @ 84 epochs (train acc: 0.8767)
# holdout test result: [0.2613, 0.903] using saved weights



# MODEL I TRANSFER LEARN VGGNET

### Plot Train and Test Accuracy

In [ ]:
model_to_plot = test_model

plt.figure(0)
plt.plot(model_to_plot.history['acc'],'b')
plt.plot(model_to_plot.history['val_acc'],'r')
plt.xticks(np.arange(1, 21, 2.0))
plt.yticks(np.arange(0.4, 1.05, 0.05))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Epoch Count")
plt.ylabel("Accuracy")
plt.title("MODEL A: Train Accuracy and Validation Accuracy")
plt.legend(['Train','Validation'])

### Plot Train and Test Loss

In [ ]:
plt.figure(0)
plt.plot(model_to_plot.history['loss'],'b')
plt.plot(model_to_plot.history['val_loss'],'r')
plt.yticks(np.arange(0, 3.0, 0.2))
plt.xticks(np.arange(1, 21, 2.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Epoch Count")
plt.ylabel("Loss")
plt.title("MODEL A: Train Loss and Validation Loss")
plt.legend(['Train','Validation'])

### Save Model and Weights

In [ ]:
# model_json = model.to_json()
# with open("5329_Model_H.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("5329_Model_H.h5")

### Prepare Confusion Matrix Function

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
label_set = np.arange(1,63)
label_text = ['0','1','2','3','4','5','6','7','8','9',
             'A','B','C','D','E','F','G','H','I','J',
             'K','L','M','N','O','P','Q','R','S','T',
             'U','V','W','X','Y','Z','a','b','c','d',
             'e','f','g','h','i','j','k','l','m','n',
             'o','p','q','r','s','t','u','v','w','x','y','z']

### Generate Classification Report

In [ ]:
y_true = y_vali
y_pred = model.predict(X_vali, verbose=1)
y_pred2 = np.argmax(y_pred, axis=1)

print(classification_report(np.argmax(y_vali,axis=1), y_pred2,target_names = label_text))

### Generate Confusion Matrix

In [ ]:
cnf_matrix = confusion_matrix(np.argmax(y_vali,axis=1),y_pred2)

# Plot normalized confusion matrix
plt.figure(figsize=(30,30))
plot_confusion_matrix(cnf_matrix, classes=label_text, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
loaded_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])